
discord music player?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot7.txt && export DEFAULT_APPLICATION=1335769252409380884 && export SECRETPATH=/Users/briancullinan/.credentials/discord7.txt && npm run import -- "discord music player"


In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')

const ALL_COMMANDS = [
  'browse',
]

// bot commands using new API, same names as above but lower-case
async function musicCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  cmdDef = {
    'name': 'browse',
    'description': 'Launch the music player',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('browse')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'browse')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = musicCommands


